In [ ]:
# KOMÓRKA 1: Montowanie Google Drive i konfiguracja Kaggle API

import os
import json
import shutil
from pathlib import Path

# Wykrycie środowiska (Colab vs lokalnie)
try:
    from google.colab import drive  # type: ignore
    _IN_COLAB = True
except Exception:
    _IN_COLAB = False

# Ustalenie katalogu roboczego (w Colab: Google Drive, lokalnie: repo)
if _IN_COLAB:
    drive.mount('/content/drive')
    DRIVE_WORKING = "/content/drive/MyDrive/mass_detection_training"
    REPO_ROOT = Path("/content")
else:
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent]
    REPO_ROOT = next((p for p in candidates if (p / "requirements.txt").exists()), cwd)
    DRIVE_WORKING = str(REPO_ROOT / "analysis" / "working")
    print("Wykryto środowisko lokalne (bez google.colab)")

os.makedirs(DRIVE_WORKING, exist_ok=True)
os.makedirs(f"{DRIVE_WORKING}/models", exist_ok=True)
os.makedirs(f"{DRIVE_WORKING}/logs", exist_ok=True)
os.makedirs(f"{DRIVE_WORKING}/splits", exist_ok=True)

print("Katalog roboczy:", DRIVE_WORKING)

# Konfiguracja Kaggle API
# Preferuj token z repo (kaggle.json w katalogu projektu), a w Colabie wspieraj też DRIVE_WORKING/kaggle.json.
kaggle_json_candidates = [
    Path(DRIVE_WORKING) / "kaggle.json",
    REPO_ROOT / "kaggle.json",
]

kaggle_json_src = next((p for p in kaggle_json_candidates if p.exists()), None)
if kaggle_json_src is None:
    raise FileNotFoundError(
        "Nie znaleziono pliku kaggle.json. Umieść go w jednym z miejsc:\n"
        f"- {kaggle_json_candidates[0]}\n"
        f"- {kaggle_json_candidates[1]}"
    )

kaggle_dir = Path(os.path.expanduser("~/.kaggle"))
kaggle_dir.mkdir(parents=True, exist_ok=True)
kaggle_json_dst = kaggle_dir / "kaggle.json"

shutil.copyfile(kaggle_json_src, kaggle_json_dst)
os.chmod(kaggle_json_dst, 0o600)

print("Skonfigurowano Kaggle API:", kaggle_json_dst)

In [ ]:
# KOMÓRKA 2: Pobieranie zbioru NIH Chest X-ray z Kaggle (45 GB)

from pathlib import Path
import os

# Jeśli masz już dane w repo (ClearScanV2/data), pomiń pobieranie z Kaggle.
local_metadata = REPO_ROOT / "data" / "metadata" / "Data_Entry_2017_v2020.csv"
local_images_dir = REPO_ROOT / "data" / "images"

if local_metadata.exists() and local_images_dir.exists():
    data_path = str(REPO_ROOT / "data")
    print("Wykryto lokalny zbiór danych w repo – pomijam pobieranie z Kaggle.")
    print(f"Ścieżka do zbioru danych: {data_path}")
else:
    import subprocess

    print("Pobieranie zbioru NIH Chest X-ray z Kaggle...")

    # Instalacja kagglehub
    try:
        import kagglehub
    except ImportError:
        print("Instaluję kagglehub...")
        subprocess.run(["pip", "install", "-q", "kagglehub"], check=True)
        import kagglehub

    # Pobranie zbioru danych za pomocą kagglehub (zalecany sposób w Colabie)
    data_path = kagglehub.dataset_download('nih-chest-xrays/data')
    print("Zbiór danych został pomyślnie pobrany!")
    print(f"Ścieżka do zbioru danych: {data_path}")

    # Sprawdzenie, czy istnieją pliki metadanych
    metadata_file = None
    for root, dirs, files in os.walk(data_path):
        if "Data_Entry_2017.csv" in files:
            metadata_file = os.path.join(root, "Data_Entry_2017.csv")
            print(f"Znaleziono plik metadanych: {metadata_file}")
            break

    if metadata_file is None:
        raise FileNotFoundError("Nie znaleziono pliku Data_Entry_2017.csv w pobranym zbiorze danych")

print("\nImport źródła danych zakończony.")

In [ ]:
# KOMÓRKA 3: Konfiguracja środowiska i instalacja pakietów

import torch
import numpy as np
import cv2

print(f"NumPy: {np.__version__}")
print(f"OpenCV: {cv2.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Dostępność CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Wykryto GPU: {torch.cuda.get_device_name(0)}")
    print(f"Pamięć GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("UWAGA: Nie wykryto GPU! (To OK lokalnie, ale trening może być wolny)")

# Instalacja pakietów (opcjonalnie)
try:
    import albumentations as _A
    print(f"Albumentations: {_A.__version__}")
except Exception:
    print("Instaluję albumentations/qudida/pyyaml...")
    import subprocess, sys
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--no-deps", "albumentations==1.4.0"], check=True)
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "qudida", "pyyaml"], check=True)

print("\nKonfiguracja środowiska zakończona.")

In [ ]:
# KOMÓRKA 4: Importy

from pathlib import Path
import json
import pandas as pd
import cv2
import gc
import time
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import (roc_auc_score, accuracy_score, confusion_matrix,
                            classification_report, roc_curve, precision_recall_curve)
from sklearn.model_selection import GroupShuffleSplit
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

print("Wszystkie moduły zostały poprawnie zaimportowane")

In [ ]:
# KOMÓRKA 5: Ścieżki i konfiguracja

# Ścieżki (w Colabie – Google Drive; lokalnie – katalog w repo)
COLAB_WORKING = Path(DRIVE_WORKING)

# Wykrywanie lokalizacji zbioru danych
print("Wyszukiwanie lokalizacji zbioru danych...")

local_metadata = REPO_ROOT / "data" / "metadata" / "Data_Entry_2017_v2020.csv"
local_images = REPO_ROOT / "data" / "images"

if local_metadata.exists() and local_images.exists():
    COLAB_DATA = local_images
    METADATA_PATH = local_metadata
    print(f"Znaleziono lokalny zbiór danych w: {COLAB_DATA}")
else:
    possible_paths = list(Path.home().glob("**/**/Data_Entry_2017.csv"))
    if not possible_paths:
        possible_paths = list(Path("/content").glob("**/Data_Entry_2017.csv"))

    if possible_paths:
        COLAB_DATA = possible_paths[0].parent
        METADATA_PATH = possible_paths[0]
        print(f"Znaleziono zbiór danych w: {COLAB_DATA}")
    else:
        raise FileNotFoundError("Nie znaleziono zbioru danych! Ponownie uruchom komórkę 2, aby go pobrać.")

# Konfiguracja modelu
MODEL_CONFIG = {
    "architecture": "DenseNet121",
    "input_channels": 1,
    "dropout": 0.4,
    "spatial_dropout": 0.2,
    "image_size": 512,
}

# Konfiguracja trenowania
TRAIN_CONFIG = {
    "stage1_epochs": 5,
    "stage2_epochs": 10,
    "stage3_epochs": 15,
    "batch_size_s1_s2": 32,
    "batch_size_s3": 16,
    "num_workers": 2,
    "base_lr": 1e-3,
    "weight_decay": 1e-4,
    "early_stopping_patience": 3,
    "grad_clip_max_norm": 1.0,
    "use_weighted_sampler": True,
}

print(f"Katalog z danymi: {COLAB_DATA}")
print(f"Katalog roboczy: {COLAB_WORKING}")
print(f"Istnienie metadanych: {METADATA_PATH.exists()}")

# Wypisz katalogi z obrazami
image_dirs = sorted([d for d in COLAB_DATA.iterdir() if d.is_dir() and d.name.startswith("images_")])
print(f"Znaleziono {len(image_dirs)} katalogów z obrazami")

# Zapisz konfigurację
with open(COLAB_WORKING / "logs/config.json", "w") as f:
    json.dump({"model": MODEL_CONFIG, "training": TRAIN_CONFIG}, f, indent=2)
print("Konfiguracja została zapisana")

In [ ]:
# KOMÓRKA 6: Klasa zbioru danych

class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, data_dir, transform=None, target_class='Mass'):
        self.dataframe = dataframe.reset_index(drop=True)
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.target_class = target_class

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Pobranie ścieżki do obrazu
        image_name = self.dataframe.iloc[idx]['Image Index']

        # Znalezienie obrazu w podkatalogach
        image_path = None
        for i in range(1, 13):
            potential_path = self.data_dir / f"images_{i:03d}" / "images" / image_name
            if potential_path.exists():
                image_path = potential_path
                break

        if image_path is None:
            raise FileNotFoundError(f"Nie znaleziono obrazu: {image_name}")

        # Wczytanie obrazu w skali szarości
        image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"Nie udało się wczytać obrazu: {image_path}")

        # Zastosowanie CLAHE z adaptacyjnymi parametrami
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        image = clahe.apply(image)

        # Zastosowanie transformacji
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        # Etykieta binarna
        finding_labels = self.dataframe.iloc[idx]['Finding Labels']
        target = 1.0 if self.target_class in finding_labels else 0.0
        target = torch.tensor([target], dtype=torch.float32)

        return image, target

print("Zdefiniowano klasę zbioru danych")


In [ ]:
# KOMÓRKA 7: Transformacje danych

train_transform = A.Compose([
    # Skalowanie dłuższego boku do docelowego rozmiaru (z zachowaniem proporcji)
    A.LongestMaxSize(max_size=MODEL_CONFIG["image_size"]),
    # Uzupełnienie do kwadratu czarnymi ramkami (lepsze niż rozciąganie obrazu)
    A.PadIfNeeded(
        min_height=MODEL_CONFIG["image_size"],
        min_width=MODEL_CONFIG["image_size"],
        border_mode=cv2.BORDER_CONSTANT,
        value=0,
        p=1.0,
    ),
    # Augmentacje
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(var_limit=(10, 50), p=0.3),
    # CoarseDropout, aby ograniczyć przeuczanie się na rogach obrazu
    A.CoarseDropout(
        max_holes=8,
        max_height=32,
        max_width=32,
        min_holes=2,
        fill_value=0,
        p=0.3,
    ),
    # Odpowiednia normalizacja dla obrazów w skali szarości
    A.Normalize(mean=[0.5], std=[0.5]),
    ToTensorV2(),
])

val_transform = A.Compose([
    # Taka sama wstępna obróbka jak w treningu (bez augmentacji)
    A.LongestMaxSize(max_size=MODEL_CONFIG["image_size"]),
    A.PadIfNeeded(
        min_height=MODEL_CONFIG["image_size"],
        min_width=MODEL_CONFIG["image_size"],
        border_mode=cv2.BORDER_CONSTANT,
        value=0,
        p=1.0,
    ),
    A.Normalize(mean=[0.5], std=[0.5]),
    ToTensorV2(),
])

print("Zdefiniowano transformacje (z zachowaniem proporcji i poprawną normalizacją)")


In [ ]:
# KOMÓRKA 8: Tworzenie podziałów z uwzględnieniem pacjentów

# Sprawdź, czy podziały już istnieją na Dysku Google
train_split_path = COLAB_WORKING / "splits/train_split.csv"
val_split_path = COLAB_WORKING / "splits/val_split.csv"
test_split_path = COLAB_WORKING / "splits/test_split.csv"

if train_split_path.exists() and val_split_path.exists() and test_split_path.exists():
    print("Podziały już istnieją na Dysku, wczytuję z plików...")
    train_df = pd.read_csv(train_split_path)
    val_df = pd.read_csv(val_split_path)
    test_df = pd.read_csv(test_split_path)
else:
    print("Tworzenie podziałów z uwzględnieniem pacjentów...")

    # Wczytanie metadanych
    metadata_df = pd.read_csv(METADATA_PATH)
    print(f"  Liczba obrazów: {len(metadata_df):,}")

    # Wyodrębnienie identyfikatorów pacjentów
    metadata_df['Patient ID'] = metadata_df['Image Index'].str.split('_').str[0]
    metadata_df['Mass'] = metadata_df['Finding Labels'].str.contains('Mass').astype(int)

    unique_patients = metadata_df['Patient ID'].nunique()
    print(f"  Liczba unikalnych pacjentów: {unique_patients:,}")

    # Podział na poziomie pacjenta
    gss_test = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
    train_val_idx, test_idx = next(gss_test.split(
        metadata_df,
        groups=metadata_df['Patient ID']
    ))

    train_val_df = metadata_df.iloc[train_val_idx].reset_index(drop=True)
    test_df = metadata_df.iloc[test_idx].reset_index(drop=True)

    gss_val = GroupShuffleSplit(n_splits=1, test_size=0.176, random_state=42)
    train_idx, val_idx = next(gss_val.split(
        train_val_df,
        groups=train_val_df['Patient ID']
    ))

    train_df = train_val_df.iloc[train_idx].reset_index(drop=True)
    val_df = train_val_df.iloc[val_idx].reset_index(drop=True)

    # Zapisanie podziałów na Dysku Google
    train_df.to_csv(train_split_path, index=False)
    val_df.to_csv(val_split_path, index=False)
    test_df.to_csv(test_split_path, index=False)

    print("  Zapisano podziały na Dysku Google")

# Statystyki podziału
print(f"\n{'='*80}")
print("PODZIAŁY Z UWZGLĘDNIENIEM PACJENTÓW")
print(f"{'='*80}")
print(f"Zbiór treningowy: {len(train_df):,} obrazów, {train_df['Patient ID'].nunique():,} pacjentów")
print(f"Zbiór walidacyjny: {len(val_df):,} obrazów, {val_df['Patient ID'].nunique():,} pacjentów")
print(f"Zbiór testowy:    {len(test_df):,} obrazów, {test_df['Patient ID'].nunique():,} pacjentów")

# Sprawdzenie braku nakładania się pacjentów między zbiorami
train_patients = set(train_df['Patient ID'])
val_patients = set(val_df['Patient ID'])
test_patients = set(test_df['Patient ID'])

overlap_train_test = train_patients & test_patients
overlap_val_test = val_patients & test_patients
overlap_train_val = train_patients & val_patients

print("\nWeryfikacja nakładania się pacjentów między zbiorami:")
print(f"  Train–Test: {len(overlap_train_test)} pacjentów (powinno być 0)")
print(f"  Val–Test:   {len(overlap_val_test)} pacjentów (powinno być 0)")
print(f"  Train–Val:  {len(overlap_train_val)} pacjentów (powinno być 0)")

if overlap_train_test or overlap_val_test or overlap_train_val:
    raise RuntimeError("Wykryto przeciek danych między zbiorami!")

print("\nPodziały danych zostały poprawnie wczytane")


In [ ]:
# KOMÓRKA 9: Tworzenie DataLoaderów (Etapy 1 i 2)


batch_size_s1_s2 = TRAIN_CONFIG["batch_size_s1_s2"]
num_workers = TRAIN_CONFIG["num_workers"]

# Utworzenie zbiorów danych
train_dataset = ChestXrayDataset(train_df, COLAB_DATA, transform=train_transform)
val_dataset = ChestXrayDataset(val_df, COLAB_DATA, transform=val_transform)
test_dataset = ChestXrayDataset(test_df, COLAB_DATA, transform=val_transform)

# Obliczenie wag klas do zbalansowanego próbkowania
train_targets = train_df['Finding Labels'].str.contains('Mass').astype(int).values
pos_count = train_targets.sum()
neg_count = len(train_targets) - pos_count
class_counts = np.array([neg_count, pos_count])
class_weights = 1.0 / class_counts
sample_weights = class_weights[train_targets]

print(f"\nRozkład klas w zbiorze treningowym:")
print(f"  Negatywne (brak zmiany Mass): {neg_count:,} ({neg_count/len(train_targets)*100:.1f}%)")
print(f"  Pozytywne (Mass):             {pos_count:,} ({pos_count/len(train_targets)*100:.1f}%)")
print(f"  Stosunek (Neg:Pos):           {neg_count/pos_count:.2f}:1")

# Utworzenie WeightedRandomSampler do zbalansowanego trenowania
if TRAIN_CONFIG.get("use_weighted_sampler", True):
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    train_loader_s1_s2 = DataLoader(
        train_dataset,
        batch_size=batch_size_s1_s2,
        sampler=sampler,
        num_workers=num_workers,
        pin_memory=True
    )
    print(f"\nUtworzono DataLoadery (Etapy 1 i 2, batch_size={batch_size_s1_s2}):")
    print("  Zastosowano WeightedRandomSampler (zbalansowane próbkowanie klas)")
else:
    train_loader_s1_s2 = DataLoader(
        train_dataset,
        batch_size=batch_size_s1_s2,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    print(f"\nUtworzono DataLoadery (Etapy 1 i 2, batch_size={batch_size_s1_s2}):")
    print("  WeightedRandomSampler wyłączony (niezbalansowane próbkowanie)")

val_loader_s1_s2 = DataLoader(
    val_dataset,
    batch_size=batch_size_s1_s2,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print(f"  Liczba batchy w treningu: {len(train_loader_s1_s2)}")
print(f"  Liczba batchy w walidacji: {len(val_loader_s1_s2)}")


In [ ]:
# KOMÓRKA 10: Definicja modelu

class MassDetectionModel(nn.Module):

    def __init__(self, pretrained=True, dropout=0.4):
        super(MassDetectionModel, self).__init__()

        self.backbone = models.densenet121(pretrained=pretrained)

        # Zmodyfikowanie pierwszej warstwy konwolucyjnej dla obrazu w skali szarości
        original_conv = self.backbone.features.conv0
        self.backbone.features.conv0 = nn.Conv2d(
            1, 64, kernel_size=7, stride=2, padding=3, bias=False
        )

        with torch.no_grad():
            self.backbone.features.conv0.weight = nn.Parameter(
                original_conv.weight.mean(dim=1, keepdim=True)
            )

        num_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()

        # Spatial Dropout2d, aby ograniczyć przeuczanie przestrzenne (artefakty w rogach)
        self.spatial_dropout = nn.Dropout2d(p=0.2)

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Dropout(p=dropout),
            nn.Linear(num_features, 1)
        )

        nn.init.kaiming_normal_(self.head[3].weight, mode='fan_out')
        nn.init.zeros_(self.head[3].bias)

    def forward(self, x):
        features = self.backbone.features(x)
        # Zastosowanie spatial dropout podczas trenowania
        if self.training:
            features = self.spatial_dropout(features)
        output = self.head(features)
        return output

    def freeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False

    def unfreeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = True

    def unfreeze_last_block(self):
        for name, param in self.backbone.named_parameters():
            if any(block in name for block in ['denseblock4', 'transition3', 'norm5']):
                param.requires_grad = True

print("Zdefiniowano architekturę modelu (ze Spatial Dropout ograniczającym artefakty w rogach)")


In [ ]:
# KOMÓRKA 11: Inicjalizacja modelu i funkcji straty

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MassDetectionModel(pretrained=True, dropout=MODEL_CONFIG["dropout"]).to(device)

print(f"Model zainicjalizowany na urządzeniu: {device}")
print(f"Łączna liczba parametrów: {sum(p.numel() for p in model.parameters()):,}")
print(f"Liczba parametrów trenowalnych: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Informacja o niezbalansowaniu klas
train_targets = train_df['Finding Labels'].str.contains('Mass').astype(int).values
pos_count = train_targets.sum()
neg_count = len(train_targets) - pos_count
imbalance_ratio = neg_count / pos_count

print(f"\nNiezbalansowanie klas: {imbalance_ratio:.1f}:1 (negatywne:pozytywne)")
print(f"  Negatywne: {neg_count:,}, Pozytywne: {pos_count:,}")
print("\nFunkcja straty: BCEWithLogitsLoss (standardowa, bez pos_weight)")
print("  Równoważenie klas realizowane przez WeightedRandomSampler (lepsze przy dużym niezbalansowaniu)")
print("  Unikamy podwójnego równoważenia (pos_weight + WeightedSampler)")

# Standardowa funkcja straty bez pos_weight (równoważenie tylko przez sampler)

criterion = nn.BCEWithLogitsLoss()

In [ ]:
# KOMÓRKA 12: Fabryka optymalizatora

def create_optimizer(model, stage, base_lr=1e-3, weight_decay=1e-4):
    if stage == 1:
        lr_backbone, lr_head = 0.0, base_lr
    elif stage == 2:
        lr_backbone, lr_head = base_lr / 10, base_lr
    elif stage == 3:
        lr_backbone, lr_head = base_lr / 100, base_lr
    else:
        raise ValueError(f"Niepoprawny etap: {stage}")

    backbone_params = []
    head_params = []

    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if 'head' in name:
            head_params.append(param)
        else:
            backbone_params.append(param)

    param_groups = [
        {'params': backbone_params, 'lr': lr_backbone, 'name': 'backbone'},
        {'params': head_params, 'lr': lr_head, 'name': 'head'}
    ]

    optimizer = optim.AdamW(param_groups, betas=(0.9, 0.999), eps=1e-8, weight_decay=weight_decay)
    return optimizer

print("Utworzono fabrykę optymalizatora")

In [ ]:
# KOMÓRKA 13: Funkcje trenowania i walidacji

def train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, stage_name=""):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_targets = []

    pbar = tqdm(train_loader, desc=f"{stage_name} Epoka {epoch} - trening")
    for images, targets in pbar:
        images = images.to(device)
        targets = targets.to(device)

        logits = model(images)
        loss = criterion(logits, targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=TRAIN_CONFIG["grad_clip_max_norm"])
        optimizer.step()

        running_loss += loss.item()
        probs = torch.sigmoid(logits).detach().cpu().numpy()
        all_preds.extend(probs.squeeze())
        all_targets.extend(targets.cpu().numpy().squeeze())

        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    avg_loss = running_loss / len(train_loader)
    auc = roc_auc_score(all_targets, all_preds)

    return avg_loss, auc


def validate(model, val_loader, criterion, device, desc="Walidacja"):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc=desc):
            images = images.to(device)
            targets = targets.to(device)

            logits = model(images)
            loss = criterion(logits, targets)

            running_loss += loss.item()
            probs = torch.sigmoid(logits).cpu().numpy()
            all_preds.extend(probs.squeeze())
            all_targets.extend(targets.cpu().numpy().squeeze())

    avg_loss = running_loss / len(val_loader)
    auc = roc_auc_score(all_targets, all_preds)

    return avg_loss, auc

print("Zdefiniowano funkcje trenowania")

In [ ]:
# KOMÓRKA 14: Klasa wczesnego zatrzymania (Early Stopping)

class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_auc = None
        self.early_stop = False

    def __call__(self, val_auc):
        if self.best_auc is None:
            self.best_auc = val_auc
        elif val_auc < self.best_auc + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_auc = val_auc
            self.counter = 0

        return self.early_stop

print("Zdefiniowano mechanizm wczesnego zatrzymania")

In [ ]:
# KOMÓRKA 15: ETAP 1 – ekstrakcja cech

print("\n" + "="*80)
print("ETAP 1: EKSTRAKCJA CECH")
print("="*80)
print("Strategia: zamrożony backbone, trenowana tylko głowa klasyfikacyjna")
print(f"Liczba epok: {TRAIN_CONFIG['stage1_epochs']}, rozmiar batcha: {TRAIN_CONFIG['batch_size_s1_s2']}")
print("="*80 + "\n")

model.freeze_backbone()
optimizer = create_optimizer(model, stage=1, base_lr=TRAIN_CONFIG["base_lr"])
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-7)
early_stopping = EarlyStopping(patience=TRAIN_CONFIG["early_stopping_patience"])

best_auc_s1 = 0.0
history_s1 = []
start_time_s1 = time.time()

for epoch in range(1, TRAIN_CONFIG['stage1_epochs'] + 1):
    train_loss, train_auc = train_one_epoch(
        model, train_loader_s1_s2, criterion, optimizer, device, epoch, "Etap 1"
)
    val_loss, val_auc = validate(model, val_loader_s1_s2, criterion, device)
    scheduler.step(val_auc)

    print(f"\nEpoka {epoch}/{TRAIN_CONFIG['stage1_epochs']}:")
    print(f"  Trening - Loss: {train_loss:.4f}, AUC: {train_auc:.4f}")
    print(f"  Walidacja - Loss: {val_loss:.4f}, AUC: {val_auc:.4f}")

    history_s1.append({
        'epoch': epoch,
        'train_loss': train_loss,
        'train_auc': train_auc,
        'val_loss': val_loss,
        'val_auc': val_auc,
    })

    if val_auc > best_auc_s1:
        best_auc_s1 = val_auc
        torch.save(model.state_dict(), COLAB_WORKING / "models/best_model_stage1.pth")
        print(f"  Zapisano najlepszy model na Dysku (AUC: {best_auc_s1:.4f})")

    if early_stopping(val_auc):
        print(f"\nWczesne zatrzymanie trenowania w epoce {epoch}")
        break

elapsed_s1 = time.time() - start_time_s1
print(f"\nEtap 1 zakończony – najlepsze AUC: {best_auc_s1:.4f}, czas: {elapsed_s1/60:.1f} min")

with open(COLAB_WORKING / "logs/history_stage1.json", "w") as f:
    json.dump(history_s1, f, indent=2)

In [ ]:
# KOMÓRKA 16: ETAP 2 – częściowe dostrajanie

print("\n" + "="*80)
print("ETAP 2: CZĘŚCIOWE DOSTRAJANIE")
print("="*80)
print("Strategia: odmrożenie ostatniego bloku sieci")
print(f"Liczba epok: {TRAIN_CONFIG['stage2_epochs']}, rozmiar batcha: {TRAIN_CONFIG['batch_size_s1_s2']}")
print("="*80 + "\n")

model.load_state_dict(torch.load(COLAB_WORKING / "models/best_model_stage1.pth"))
model.unfreeze_last_block()

optimizer = create_optimizer(model, stage=2, base_lr=TRAIN_CONFIG["base_lr"])
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-7)
early_stopping = EarlyStopping(patience=TRAIN_CONFIG["early_stopping_patience"])

best_auc_s2 = 0.0
history_s2 = []
start_time_s2 = time.time()

for epoch in range(1, TRAIN_CONFIG['stage2_epochs'] + 1):
    train_loss, train_auc = train_one_epoch(
        model, train_loader_s1_s2, criterion, optimizer, device, epoch, "Etap 2"
)
    val_loss, val_auc = validate(model, val_loader_s1_s2, criterion, device)
    scheduler.step(val_auc)

    print(f"\nEpoka {epoch}/{TRAIN_CONFIG['stage2_epochs']}:")
    print(f"  Trening - Loss: {train_loss:.4f}, AUC: {train_auc:.4f}")
    print(f"  Walidacja - Loss: {val_loss:.4f}, AUC: {val_auc:.4f}")

    history_s2.append({
        'epoch': epoch,
        'train_loss': train_loss,
        'train_auc': train_auc,
        'val_loss': val_loss,
        'val_auc': val_auc,
    })

    if val_auc > best_auc_s2:
        best_auc_s2 = val_auc
        torch.save(model.state_dict(), COLAB_WORKING / "models/best_model_stage2.pth")
        print(f"  Zapisano najlepszy model na Dysku (AUC: {best_auc_s2:.4f})")

    if early_stopping(val_auc):
        print(f"\nWczesne zatrzymanie trenowania w epoce {epoch}")
        break

elapsed_s2 = time.time() - start_time_s2
print(f"\nEtap 2 zakończony – najlepsze AUC: {best_auc_s2:.4f}, czas: {elapsed_s2/60:.1f} min")

with open(COLAB_WORKING / "logs/history_stage2.json", "w") as f:
    json.dump(history_s2, f, indent=2)

In [ ]:
# KOMÓRKA 17: DataLoadery dla Etapu 3

batch_size_s3 = TRAIN_CONFIG["batch_size_s3"]

# Użycie tego samego WeightedRandomSampler dla Etapu 3
if TRAIN_CONFIG.get("use_weighted_sampler", True):
    train_loader_s3 = DataLoader(
        train_dataset,
        batch_size=batch_size_s3,
        sampler=sampler,
        num_workers=num_workers,
        pin_memory=True
    )
    print("Etap 3 wykorzystuje WeightedRandomSampler (zbalansowane próbkowanie)")
else:
    train_loader_s3 = DataLoader(
        train_dataset,
        batch_size=batch_size_s3,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

val_loader_s3 = DataLoader(
    val_dataset,
    batch_size=batch_size_s3,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print(f"Utworzono DataLoadery dla Etapu 3 (batch_size={batch_size_s3}):")
print(f"  Liczba batchy w treningu: {len(train_loader_s3)}")
print(f"  Liczba batchy w walidacji: {len(val_loader_s3)}")


In [ ]:
# KOMÓRKA 18: ETAP 3 – pełne dostrajanie

print("\n" + "="*80)
print("ETAP 3: PEŁNE DOSTRAJANIE")
print("="*80)
print("Strategia: odmrożenie całego backbone")
print(f"Liczba epok: {TRAIN_CONFIG['stage3_epochs']}, rozmiar batcha: {TRAIN_CONFIG['batch_size_s3']}")
print("="*80 + "\n")

torch.cuda.empty_cache()
gc.collect()

model.load_state_dict(torch.load(COLAB_WORKING / "models/best_model_stage2.pth"))
model.unfreeze_backbone()

optimizer = create_optimizer(model, stage=3, base_lr=TRAIN_CONFIG["base_lr"])
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-7)
early_stopping = EarlyStopping(patience=TRAIN_CONFIG["early_stopping_patience"])

best_auc_s3 = 0.0
history_s3 = []
start_time_s3 = time.time()

for epoch in range(1, TRAIN_CONFIG['stage3_epochs'] + 1):
    torch.cuda.empty_cache()
    gc.collect()

    train_loss, train_auc = train_one_epoch(
        model, train_loader_s3, criterion, optimizer, device, epoch, "Etap 3"
)
    val_loss, val_auc = validate(model, val_loader_s3, criterion, device)
    scheduler.step(val_auc)

    print(f"\nEpoka {epoch}/{TRAIN_CONFIG['stage3_epochs']}:")
    print(f"  Trening - Loss: {train_loss:.4f}, AUC: {train_auc:.4f}")
    print(f"  Walidacja - Loss: {val_loss:.4f}, AUC: {val_auc:.4f}")

    history_s3.append({
        'epoch': epoch,
        'train_loss': train_loss,
        'train_auc': train_auc,
        'val_loss': val_loss,
        'val_auc': val_auc,
    })

    if val_auc > best_auc_s3:
        best_auc_s3 = val_auc
        torch.save(model.state_dict(), COLAB_WORKING / "models/best_model_stage3.pth")
        print(f"  Zapisano najlepszy model na Dysku (AUC: {best_auc_s3:.4f})")

    if early_stopping(val_auc):
        print(f"\nWczesne zatrzymanie trenowania w epoce {epoch}")
        break

elapsed_s3 = time.time() - start_time_s3
print(f"\nEtap 3 zakończony – najlepsze AUC: {best_auc_s3:.4f}, czas: {elapsed_s3/60:.1f} min")

with open(COLAB_WORKING / "logs/history_stage3.json", "w") as f:
    json.dump(history_s3, f, indent=2)

history_combined = {
    'stage1': history_s1,
    'stage2': history_s2,
    'stage3': history_s3,
    'summary': {
        'best_auc_stage1': best_auc_s1,
        'best_auc_stage2': best_auc_s2,
        'best_auc_stage3': best_auc_s3,
        'total_time_minutes': (elapsed_s1 + elapsed_s2 + elapsed_s3) / 60,
    }
}
with open(COLAB_WORKING / "logs/training_history_complete.json", "w") as f:
    json.dump(history_combined, f, indent=2)

print("\n" + "="*80)
print("TRENOWANIE ZAKOŃCZONE")
print("="*80)
print(f"Etap 1 – najlepsze AUC: {best_auc_s1:.4f} (czas: {elapsed_s1/60:.1f} min)")
print(f"Etap 2 – najlepsze AUC: {best_auc_s2:.4f} (czas: {elapsed_s2/60:.1f} min)")
print(f"Etap 3 – najlepsze AUC: {best_auc_s3:.4f} (czas: {elapsed_s3/60:.1f} min)")
print(f"Łączny czas: {(elapsed_s1 + elapsed_s2 + elapsed_s3)/60:.1f} min")
print(f"\nWszystkie checkpointy zapisano na Dysku Google: {COLAB_WORKING}")

---

# EVALUATION PIPELINE

Comprehensive test set evaluation with metrics, visualizations, and Grad-CAM.

In [ ]:
# KOMÓRKA 19: Ewaluacja na zbiorze testowym

print("\n" + "="*80)
print("EWALUACJA NA ZBIORZE TESTOWYM")
print("="*80 + "\n")

test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

model.load_state_dict(torch.load(COLAB_WORKING / "models/best_model_stage3.pth"))
print(f"Wczytano najlepszy model z Etapu 3 (AUC walidacyjne: {best_auc_s3:.4f})")

start_time_eval = time.time()
model.eval()
all_targets = []
all_probs = []

with torch.no_grad():
    for images, targets in tqdm(test_loader, desc="Ewaluacja na zbiorze testowym"):
        images = images.to(device)
        targets = targets.to(device)
        logits = model(images)
        probs = torch.sigmoid(logits).cpu().numpy().squeeze()
        all_probs.extend(probs)
        all_targets.extend(targets.cpu().numpy().squeeze())

all_targets = np.array(all_targets)
all_probs = np.array(all_probs)

eval_time = time.time() - start_time_eval
print(f"Ewaluacja zakończona (czas: {eval_time/60:.1f} min)")

In [ ]:
# KOMÓRKA 20: Podstawowe metryki

threshold_default = 0.5
preds_default = (all_probs >= threshold_default).astype(int)

auc_test = roc_auc_score(all_targets, all_probs)
acc_test = accuracy_score(all_targets, preds_default)
cm_default = confusion_matrix(all_targets, preds_default)
report_default = classification_report(all_targets, preds_default,
                                       target_names=["No Finding", "Mass"], digits=4)

print("\n" + "="*80)
print(f"WYNIKI NA ZBIORZE TESTOWYM (Próg={threshold_default})")
print("="*80)
print(f"AUC-ROC: {auc_test:.4f}")
print(f"Accuracy: {acc_test:.4f}")
print("\nMacierz pomyłek:")
print(cm_default)
print("\nRaport klasyfikacji:")
print(report_default)

tn, fp, fn, tp = cm_default.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print("\nMetryki kliniczne:")
print(f"  Czułość (Sensitivity): {sensitivity:.4f}")
print(f"  Swoistość (Specificity): {specificity:.4f}")
print(f"  PPV: {ppv:.4f}")
print(f"  NPV: {npv:.4f}")

In [ ]:
# KOMÓRKA 21: Optymalizacja progu

print("\n" + "="*80)
print("OPTYMALIZACJA PROGU DECYZYJNEGO")
print("="*80 + "\n")

precisions, recalls, thresholds_pr = precision_recall_curve(all_targets, all_probs)

target_recall = 0.70
idx_recall = np.where(recalls >= target_recall)[0]
if len(idx_recall) > 0:
    optimal_threshold_recall = thresholds_pr[idx_recall[-1]]
    optimal_precision_recall = precisions[idx_recall[-1]]
else:
    optimal_threshold_recall = 0.5
    optimal_precision_recall = 0.0

f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)
idx_f1 = np.argmax(f1_scores)
optimal_threshold_f1 = thresholds_pr[idx_f1]

fpr_roc, tpr_roc, thresholds_roc = roc_curve(all_targets, all_probs)
youdens = tpr_roc - fpr_roc
idx_youden = np.argmax(youdens)
optimal_threshold_youden = thresholds_roc[idx_youden]

print("Optymalne progi:")
print(f"  1. Czułość ≥ 70%:  {optimal_threshold_recall:.4f} (Precyzja: {optimal_precision_recall:.4f})")
print(f"  2. Maksymalny F1-score:  {optimal_threshold_f1:.4f} (F1: {f1_scores[idx_f1]:.4f})")
print(f"  3. Wskaźnik Youdena: {optimal_threshold_youden:.4f}")

preds_optimal = (all_probs >= optimal_threshold_recall).astype(int)
cm_optimal = confusion_matrix(all_targets, preds_optimal)
report_optimal = classification_report(all_targets, preds_optimal,
                                      target_names=["No Finding", "Mass"], digits=4)

print(f"\nWyniki dla optymalnego progu ({optimal_threshold_recall:.4f}):")
print("\nMacierz pomyłek:")
print(cm_optimal)
print("\nRaport klasyfikacji:")
print(report_optimal)

threshold_data = {
    "default_threshold": float(threshold_default),
    "optimal_threshold_recall_70": float(optimal_threshold_recall),
    "optimal_threshold_f1": float(optimal_threshold_f1),
    "optimal_threshold_youden": float(optimal_threshold_youden),
}

with open(COLAB_WORKING / "logs/threshold_optimization.json", "w") as f:
    json.dump(threshold_data, f, indent=2)

In [ ]:
# KOMÓRKA 22: Zapis metryk i predykcji

test_metrics = {
    "auc_roc": float(auc_test),
    "accuracy": float(acc_test),
    "clinical_metrics": {
        "sensitivity": float(sensitivity),
        "specificity": float(specificity),
        "ppv": float(ppv),
        "npv": float(npv)
    }
}

with open(COLAB_WORKING / "logs/test_metrics.json", "w") as f:
    json.dump(test_metrics, f, indent=2)

predictions_df = pd.DataFrame({
    'image_index': test_df['Image Index'].values,
    'patient_id': test_df['Patient ID'].values,
    'true_label': all_targets,
    'predicted_prob': all_probs,
    'predicted_class_threshold_05': preds_default,
    'predicted_class_optimal': preds_optimal
})
predictions_df.to_csv(COLAB_WORKING / "logs/all_predictions.csv", index=False)

print("Metryki zapisano na Dysku Google")

In [ ]:
# KOMÓRKA 23: Krzywa ROC

plt.figure(figsize=(7, 7))
plt.plot(fpr_roc, tpr_roc, linewidth=2, label=f'Krzywa ROC (AUC = {auc_test:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Losowy klasyfikator')
plt.scatter([fpr_roc[idx_youden]], [tpr_roc[idx_youden]], color='red', s=100,
            zorder=5, label=f"Punkt Youdena (próg={optimal_threshold_youden:.3f})")
plt.xlabel('Odsetek fałszywie pozytywnych (FPR)', fontsize=12)
plt.ylabel('Odsetek prawdziwie pozytywnych (TPR)', fontsize=12)
plt.title('Krzywa ROC – detekcja zmian typu Mass', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(COLAB_WORKING / "logs/roc_curve.png", dpi=150, bbox_inches='tight')
plt.show()

print("Krzywą ROC zapisano na Dysku Google")

In [ ]:
# KOMÓRKA 24: Krzywa precyzja–czułość

plt.figure(figsize=(7, 7))
plt.plot(recalls, precisions, linewidth=2, label='Krzywa precyzja–czułość')
plt.scatter([target_recall], [optimal_precision_recall], color='red', s=100,
            zorder=5, label=f'Czułość ≥ 70% (próg={optimal_threshold_recall:.3f})')
plt.axhline(y=optimal_precision_recall, color='red', linestyle='--', alpha=0.3)
plt.axvline(x=target_recall, color='red', linestyle='--', alpha=0.3)
plt.xlabel('Czułość (Recall)', fontsize=12)
plt.ylabel('Precyzja (Precision)', fontsize=12)
plt.title('Krzywa precyzja–czułość', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(COLAB_WORKING / "logs/precision_recall_curve.png", dpi=150, bbox_inches='tight')
plt.show()

print("Krzywą precyzja–czułość zapisano na Dysku Google")

In [ ]:
# KOMÓRKA 25: Macierze pomyłek

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(cm_default, annot=True, fmt='d', cmap='Blues',
            xticklabels=["No Finding", "Mass"],
            yticklabels=["No Finding", "Mass"],
            ax=axes[0], cbar=True, annot_kws={"fontsize": 12})
axes[0].set_xlabel('Klasa przewidziana', fontsize=11)
axes[0].set_ylabel('Klasa rzeczywista', fontsize=11)
axes[0].set_title(f'Próg={threshold_default}', fontsize=12, fontweight='bold')

sns.heatmap(cm_optimal, annot=True, fmt='d', cmap='Greens',
            xticklabels=["No Finding", "Mass"],
            yticklabels=["No Finding", "Mass"],
            ax=axes[1], cbar=True, annot_kws={"fontsize": 12})
axes[1].set_xlabel('Klasa przewidziana', fontsize=11)
axes[1].set_ylabel('Klasa rzeczywista', fontsize=11)
axes[1].set_title(f'Próg={optimal_threshold_recall:.3f}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(COLAB_WORKING / "logs/confusion_matrices.png", dpi=150, bbox_inches='tight')
plt.show()

print("Macierze pomyłek zapisano na Dysku Google")

In [ ]:
# KOMÓRKA 26: Wizualizacja Grad-CAM

def generate_gradcam(model, image_tensor, device='cuda'):
    model.eval()
    image_tensor = image_tensor.unsqueeze(0).to(device)
    image_tensor.requires_grad = True

    features = None
    gradients = None

    def forward_hook(module, input, output):
        nonlocal features
        features = output

    def backward_hook(module, grad_input, grad_output):
        nonlocal gradients
        gradients = grad_output[0]

    target_layer = model.backbone.features.denseblock4
    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_full_backward_hook(backward_hook)

    output = model(image_tensor)

    model.zero_grad()
    class_loss = output[0, 0]
    class_loss.backward()

    forward_handle.remove()
    backward_handle.remove()

    pooled_grads = torch.mean(gradients, dim=[0, 2, 3])
    for i in range(features.shape[1]):
        features[:, i, :, :] *= pooled_grads[i]

    heatmap = features.mean(dim=1).squeeze().cpu().detach().numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-8)

    return heatmap


print("Wizualizacja Grad-CAM")
images_sample, targets_sample = next(iter(test_loader))

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for i in range(min(6, len(images_sample))):
    heatmap = generate_gradcam(model, images_sample[i], device=device)
    heatmap_resized = cv2.resize(heatmap, (MODEL_CONFIG["image_size"], MODEL_CONFIG["image_size"]))

    img = images_sample[i].cpu().squeeze().numpy()
    axes[i].imshow(img, cmap='gray')
    axes[i].imshow(heatmap_resized, cmap='jet', alpha=0.4)
    axes[i].set_title(f"Etykieta: {'Mass' if targets_sample[i].item() == 1 else 'No Finding'}", fontsize=10)
    axes[i].axis('off')

plt.suptitle('Grad-CAM – obszary uwagi modelu', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(COLAB_WORKING / "logs/gradcam_examples.png", dpi=150, bbox_inches='tight')
plt.show()

print("Wizualizacje Grad-CAM zapisano na Dysku Google")

In [ ]:
# KOMÓRKA 27: Podsumowanie końcowe

print("\n" + "="*80)
print("KOMPLETNY PIPELINE ZAKOŃCZONY")
print("="*80)

summary = {
    "training": {
        "stage1_best_val_auc": best_auc_s1,
        "stage2_best_val_auc": best_auc_s2,
        "stage3_best_val_auc": best_auc_s3,
        "total_time_minutes": (elapsed_s1 + elapsed_s2 + elapsed_s3) / 60
    },
    "evaluation": {
        "test_auc": auc_test,
        "test_accuracy": acc_test,
        "threshold_optimal": optimal_threshold_recall
    },
    "splits": {
        "train_images": len(train_df),
        "val_images": len(val_df),
        "test_images": len(test_df)
    }
}

with open(COLAB_WORKING / "logs/final_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\nPODSUMOWANIE TRENOWANIA:")
print(f"  Etap 1: {best_auc_s1:.4f} ({elapsed_s1/60:.1f} min)")
print(f"  Etap 2: {best_auc_s2:.4f} ({elapsed_s2/60:.1f} min)")
print(f"  Etap 3: {best_auc_s3:.4f} ({elapsed_s3/60:.1f} min)")
print(f"  Łącznie: {(elapsed_s1 + elapsed_s2 + elapsed_s3)/60:.1f} min")

print("\nPODSUMOWANIE EWALUACJI TESTOWEJ:")
print(f"  AUC na zbiorze testowym: {auc_test:.4f}")
print(f"  Accuracy na zbiorze testowym: {acc_test:.4f}")

print(f"\nWszystkie wyniki zapisano na Dysku Google: {COLAB_WORKING}")
print("\nPLIKI NA DYSKU:")
print("/MyDrive/mass_detection_training/models/")
print("- best_model_stage1.pth")
print("- best_model_stage2.pth")
print("- best_model_stage3.pth")
print("/MyDrive/mass_detection_training/logs/")
print("- training_history_complete.json")
print("- test_metrics.json")
print("- all_predictions.csv")
print("- pliki PNG (wizualizacje)")
print("/MyDrive/mass_detection_training/splits/")
print("- train_split.csv, val_split.csv, test_split.csv")
print("="*80)

---

## Instructions for Google Colab

### First Time Setup:
1. Change Runtime: Runtime → Change runtime type → T4 GPU
2. Run Cell 1: Mount Google Drive (authorize access)
3. Run Cell 2: Download dataset (20–40 min, ~45 GB)
4. Run Cells 3–27: Complete training pipeline

### Resume Training (if session times out):
1. Run Cell 1 (mount Drive)
2. Skip Cell 2 (dataset already downloaded to `/content/data/`)
3. Run Cell 3–8 (setup + load existing splits from Drive)
4. Load checkpoint from Drive:
   ```python
   model.load_state_dict(torch.load(COLAB_WORKING / "models/best_model_stage2.pth"))
   ```
5. Continue from desired stage

### Key Features:
- Persistent Storage: Checkpoints saved to Google Drive (survives session timeouts)
- No Manual Upload: Dataset downloads automatically via Kaggle API
- Same Splits: Patient-stratified splits saved to Drive (deterministic)
- Resume Capability: Can restart from any stage using Drive checkpoints

### Tips:
- Colab free tier: ~12h runtime limit (enough for full training)
- Dataset stays in `/content/data/` for current session
- Checkpoints in Drive persist over time
- Re-run Cell 2 only if `/content/data/` is deleted (new session)